# 05 - Test Anonymisation PDF avec PyMuPDF

Objectif : Remplacer le nom de l'élève dans le PDF **avant** envoi à Mistral OCR.

## Flow
```
PDF original → pdfplumber (extraire nom) → PyMuPDF (remplacer nom) → PDF anonymisé → Mistral OCR
```

In [1]:
import sys
from pathlib import Path

# Auto-détecter project_root
current = Path.cwd()
while current != current.parent:
    if (current / "pyproject.toml").exists():
        project_root = current
        break
    current = current.parent

sys.path.insert(0, str(project_root))

DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"

# PDF de test avec un vrai nom (Marie Dupont)
test_pdf = RAW_DIR / "ELEVE_TEST_marie_dupont.pdf"
print(f"PDF de test: {test_pdf}")
print(f"Existe: {test_pdf.exists()}")

PDF de test: c:\Users\Florent\Documents\data_science\chiron\data\raw\ELEVE_TEST_marie_dupont.pdf
Existe: True


## 1. Extraire le nom de l'élève avec pdfplumber

In [2]:
import re

import pdfplumber


def extract_eleve_name(pdf_path: Path) -> dict | None:
    """Extrait le nom de l'élève depuis le PDF.

    Recherche le pattern "Élève : <nom>" (ou "Elève", case insensitive) et capture
    tout le texte jusqu'à la fin de la ligne. Fonctionne pour les prénoms composés
    et noms multiples tant qu'ils sont sur la même ligne que "Élève :".

    Args:
        pdf_path: Chemin vers le fichier PDF.

    Returns:
        Dict avec 'nom_complet' et 'texte_complet', ou None si non trouvé.
    """
    with pdfplumber.open(pdf_path) as pdf:
        text_complet = ""
        for page in pdf.pages:
            text_complet += (page.extract_text() or "") + "\n"

        # Pattern: "Élève : NOM" ou "Elève : NOM" (case insensitive)
        match = re.search(r"[ÉE]l[èe]ve\s*:\s*([^\n]+)", text_complet, re.IGNORECASE)
        if match:
            return {
                "nom_complet": match.group(1).strip(),
                "texte_complet": text_complet,
            }
    return None

In [3]:
# Test
eleve_info = extract_eleve_name(test_pdf)
print(f"Nom extrait de l'en-tête: {eleve_info['nom_complet']}")

Nom extrait de l'en-tête: Marie Dupont


## 2. NER : Détecter tous les noms de personnes avec spaCy

On utilise le NER (Named Entity Recognition) pour trouver toutes les mentions de personnes dans le texte, puis on compare avec le nom extrait de l'en-tête.

In [4]:
# Charger les 3 modèles NER pour comparaison

# 1. spaCy (modèle statistique)
import spacy

try:
    nlp_spacy = spacy.load("fr_core_news_lg")
    print("✓ spaCy chargé: fr_core_news_lg")
except OSError:
    print("✗ spaCy non disponible")
    nlp_spacy = None

# 2. CamemBERT-NER (ancien, 2020)
from transformers import pipeline

try:
    nlp_camembert_v1 = pipeline(
        "ner",
        model="Jean-Baptiste/camembert-ner",
        aggregation_strategy="simple"
    )
    print("✓ CamemBERT v1 chargé: Jean-Baptiste/camembert-ner")
except Exception as e:
    print(f"✗ CamemBERT v1 non disponible: {e}")
    nlp_camembert_v1 = None

# 3. CamemBERTav2-NER (nouveau, 2024, F1=94%)
try:
    nlp_camembert_v2 = pipeline(
        "ner",
        model="almanach/camembertav2-base-ftb-ner",
        aggregation_strategy="simple"
    )
    print("✓ CamemBERTav2 chargé: almanach/camembertav2-base-ftb-ner")
except Exception as e:
    print(f"✗ CamemBERTav2 non disponible: {e}")
    nlp_camembert_v2 = None

✓ spaCy chargé: fr_core_news_lg


c:\Users\Florent\Documents\data_science\chiron\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


✓ CamemBERT v1 chargé: Jean-Baptiste/camembert-ner


Device set to use cpu


✓ CamemBERTav2 chargé: almanach/camembertav2-base-ftb-ner


In [5]:
# Comparer les 3 modèles NER
import time

text = eleve_info["texte_complet"] if eleve_info else ""

print("=" * 70)
print("COMPARAISON DES 3 MODÈLES NER")
print("=" * 70)

results = {}

# Fonction pour découper en chunks par phrases (évite de couper les noms)
def split_into_chunks(text: str, max_chars: int = 2000) -> list[str]:
    """Découpe le texte en chunks en respectant les fins de phrases.

    Args:
        text: Texte à découper
        max_chars: Taille max par chunk (en caractères)
            - CamemBERT v1 (512 tokens) : ~2000 chars
            - CamemBERTav2 (1024 tokens) : ~4000 chars

    Returns:
        Liste de chunks
    """
    import re
    sentences = re.split(r'(?<=[.!?])\s+', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_chars:
            current_chunk += sentence + " "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks if chunks else [text]

# Labels pour les personnes selon le modèle
PERSON_LABELS = {"PER", "PERSON", "I-PER", "B-PER"}

def is_person_entity(entity_group: str) -> bool:
    """Vérifie si le label correspond à une personne."""
    return entity_group.upper() in PERSON_LABELS or "PER" in entity_group.upper()

# 1. spaCy (pas de limite de tokens)
if nlp_spacy and text:
    start = time.perf_counter()
    doc = nlp_spacy(text)
    elapsed = time.perf_counter() - start
    personnes = [ent.text for ent in doc.ents if ent.label_ == "PER"]
    results["spaCy"] = {"personnes": personnes, "time": elapsed}

    print(f"\n[1] spaCy fr_core_news_lg ({elapsed:.3f}s)")
    print(f"    Personnes détectées: {len(personnes)}")
    for p in set(personnes):
        print(f"      - '{p}' ({personnes.count(p)}x)")

# 2. CamemBERT v1 (512 tokens max) - label: PER
if nlp_camembert_v1 and text:
    start = time.perf_counter()
    chunks = split_into_chunks(text, max_chars=2000)
    personnes = []
    for chunk in chunks:
        entities = nlp_camembert_v1(chunk)
        personnes.extend([e["word"] for e in entities if is_person_entity(e["entity_group"])])
    elapsed = time.perf_counter() - start
    results["CamemBERT_v1"] = {"personnes": personnes, "time": elapsed}

    print(f"\n[2] CamemBERT v1 - Jean-Baptiste/camembert-ner ({elapsed:.3f}s)")
    print(f"    Personnes détectées: {len(personnes)}")
    for p in set(personnes):
        print(f"      - '{p}' ({personnes.count(p)}x)")

# 3. CamemBERTav2 (1024 tokens max) - label: Person
if nlp_camembert_v2 and text:
    start = time.perf_counter()
    chunks = split_into_chunks(text, max_chars=4000)
    personnes = []
    for chunk in chunks:
        entities = nlp_camembert_v2(chunk)
        personnes.extend([e["word"] for e in entities if is_person_entity(e["entity_group"])])
    elapsed = time.perf_counter() - start
    results["CamemBERTav2"] = {"personnes": personnes, "time": elapsed}

    print(f"\n[3] CamemBERTav2 - almanach/camembertav2-base-ftb-ner ({elapsed:.3f}s)")
    print(f"    Personnes détectées: {len(personnes)}")
    for p in set(personnes):
        print(f"      - '{p}' ({personnes.count(p)}x)")

# Résumé
print("\n" + "=" * 70)
print("RÉSUMÉ")
print("=" * 70)
print(f"{'Modèle':<25} {'Détections':<12} {'Temps':<10} {'Faux positifs'}")
print("-" * 70)
for name, data in results.items():
    unique = set(data["personnes"])
    faux_pos = [p for p in unique if "marie" not in p.lower() and "dupont" not in p.lower()]
    print(f"{name:<25} {len(data['personnes']):<12} {data['time']:.3f}s     {faux_pos if faux_pos else 'Aucun'}")

# Sélection automatique du meilleur modèle (CamemBERT v1 = meilleur compromis)
if nlp_camembert_v1 and results.get("CamemBERT_v1", {}).get("personnes"):
    personnes_detectees = list(set(results["CamemBERT_v1"]["personnes"]))
    print("\n→ Modèle sélectionné: CamemBERT v1 (meilleur compromis vitesse/précision)")
elif nlp_camembert_v2 and results.get("CamemBERTav2", {}).get("personnes"):
    personnes_detectees = list(set(results["CamemBERTav2"]["personnes"]))
    print("\n→ Modèle sélectionné: CamemBERTav2")
else:
    personnes_detectees = list(set(results.get("spaCy", {}).get("personnes", [])))
    print("\n→ Modèle sélectionné: spaCy (attention: faux positifs possibles)")

print(f"   Variantes uniques: {personnes_detectees}")

COMPARAISON DES 3 MODÈLES NER

[1] spaCy fr_core_news_lg (0.121s)
    Personnes détectées: 10
      - 'Matière Élève' (1x)
      - 'Physique-Chimie' (1x)
      - 'Marie Dupont
Genre' (1x)
      - 'Marie Dupont' (7x)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



[2] CamemBERT v1 - Jean-Baptiste/camembert-ner (1.293s)
    Personnes détectées: 8
      - 'Marie Dupont' (8x)

[3] CamemBERTav2 - almanach/camembertav2-base-ftb-ner (1.602s)
    Personnes détectées: 8
      - 'Marie Dupont' (8x)

RÉSUMÉ
Modèle                    Détections   Temps      Faux positifs
----------------------------------------------------------------------
spaCy                     10           0.121s     ['Matière Élève', 'Physique-Chimie']
CamemBERT_v1              8            1.293s     Aucun
CamemBERTav2              8            1.602s     Aucun

→ Modèle sélectionné: CamemBERT v1 (meilleur compromis vitesse/précision)
   Variantes uniques: ['Marie Dupont']


In [6]:
def find_eleve_variants(nom_entete: str, personnes_ner: list[str]) -> list[str]:
    """Compare le nom de l'en-tête avec les entités NER pour trouver les variantes.

    Args:
        nom_entete: Nom extrait de "Élève : XXX"
        personnes_ner: Liste des personnes détectées par NER

    Returns:
        Liste des variantes du nom de l'élève à anonymiser (triées par longueur desc)
    """
    variants = set()

    # Parser le nom de l'en-tête
    nom_entete_clean = nom_entete.strip()
    nom_parts = [p.lower() for p in nom_entete_clean.split()]

    for personne in set(personnes_ner):
        personne_clean = personne.strip()
        if not personne_clean:
            continue

        personne_lower = personne_clean.lower()
        personne_parts = personne_lower.split()

        # Match si au moins une partie correspond au nom de l'en-tête
        if any(part in nom_parts for part in personne_parts):
            variants.add(personne_clean)

    # Toujours inclure le nom complet de l'en-tête
    variants.add(nom_entete_clean)

    # Trier par longueur décroissante (remplacer les plus longs d'abord)
    return sorted(variants, key=len, reverse=True)


In [7]:
personnes_detectees = list(set(results["CamemBERT_v1"]["personnes"]))
print(personnes_detectees)

['Marie Dupont']


In [8]:
# Test
if personnes_detectees and eleve_info:
    variants = find_eleve_variants(eleve_info["nom_complet"], personnes_detectees)
    print(f"Nom de l'en-tête: '{eleve_info['nom_complet']}'")
    print(f"\nVariantes à anonymiser ({len(variants)}):")
    for v in variants:
        print(f"  - '{v}'")

Nom de l'en-tête: 'Marie Dupont'

Variantes à anonymiser (1):
  - 'Marie Dupont'


## 3. Visualiser le PDF original

In [9]:
import fitz  # PyMuPDF

# Ouvrir le PDF
doc = fitz.open(test_pdf)
page = doc[0]

print(f"Nombre de pages: {len(doc)}")
print(f"Taille page: {page.rect.width} x {page.rect.height}")

# Extraire le texte pour vérification
text = page.get_text()
print(f"\nTexte complet ({len(text)} chars):")
print(text[:1000])

# Vérifier si le nom est présent dans le texte extrait
nom_recherche = eleve_info["nom_complet"] if eleve_info else "Marie Dupont"
if nom_recherche in text:
    pos = text.find(nom_recherche)
    print(f"\n✅ '{nom_recherche}' trouvé à la position {pos}/{len(text)}")
    # Montrer le contexte
    start = max(0, pos - 30)
    end = min(len(text), pos + len(nom_recherche) + 30)
    print(f"   Contexte: ...{repr(text[start:end])}...")
else:
    print(f"\n⚠️ '{nom_recherche}' NON trouvé dans le texte extrait")
    for part in nom_recherche.split():
        if part in text:
            print(f"   - '{part}' trouvé")
        else:
            print(f"   - '{part}' NON trouvé")

# Analyser les polices utilisées dans le PDF
print("\n" + "=" * 50)
print("ANALYSE DES POLICES")
print("=" * 50)

# Extraire avec info de police (dict format)
blocks = page.get_text("dict")["blocks"]
fonts_used = {}

for block in blocks:
    if "lines" in block:
        for line in block["lines"]:
            for span in line["spans"]:
                font = span["font"]
                size = span["size"]
                text_span = span["text"].strip()
                if text_span:
                    key = f"{font} ({size:.1f}pt)"
                    if key not in fonts_used:
                        fonts_used[key] = []
                    fonts_used[key].append(text_span[:50])

print("\nPolices utilisées:")
for font_key, texts in sorted(fonts_used.items(), key=lambda x: x[0]):
    print(f"\n{font_key}:")
    for t in texts[:3]:
        print(f"  - '{t}'")
    if len(texts) > 3:
        print(f"  ... et {len(texts) - 3} autres")



Nombre de pages: 1
Taille page: 595.2755737304688 x 841.8897705078125

Texte complet (2036 chars):
College Test
BULLETIN SCOLAIRE
Élève : Marie Dupont
Genre : Fille
Absences : 4 demi-journées (justifiées)
Engagements : Déléguée titulaire
Matière
Élève / Classe
Appréciation
Anglais LV1
15.21 / 10.83
Bons résultats. Marie Dupont fournit un travail régulier et sérieux à la maison tout comme
en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi!
Arts Plastiques
15.00 / 14.92
Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentrée.
EPS
8.00 / 12.79
Bilan très insuffisant, Marie Dupont n'a pas réussi à s'orienter avec efficacité. Elle a
beaucoup marché et s'est dispersée avec son groupe. Son travail a manqué de sérieux
et d'implication dans les défis à relever en course d'orientation. De sincères efforts sont
attendus au prochain trimestre.
Éducation Musicale
13.00 / 9.53
C'est très bien quand vous voulez. Votre investissement doit être régulier.
Espag

In [10]:
# Chercher les occurrences du nom dans le PDF
nom_a_chercher = eleve_info["nom_complet"] if eleve_info else "ELEVE_A"
print(f"Recherche de: '{nom_a_chercher}'")

doc = fitz.open(test_pdf)
for page_num, page in enumerate(doc):
    instances = page.search_for(nom_a_chercher)
    print(f"\nPage {page_num + 1}: {len(instances)} occurrence(s)")
    for i, rect in enumerate(instances):
        print(f"  [{i+1}] Position: {rect}")

Recherche de: 'Marie Dupont'

Page 1: 8 occurrence(s)
  [1] Position: Rect(83.53968811035156, 120.273681640625, 143.5596923828125, 134.01368713378906)
  [2] Position: Rect(280.64276123046875, 213.59686279296875, 328.5696105957031, 224.5888671875)
  [3] Position: Rect(301.71697998046875, 255.59686279296875, 349.73297119140625, 266.5888671875)
  [4] Position: Rect(380.4129943847656, 343.59686279296875, 428.4289855957031, 354.5888671875)
  [5] Position: Rect(279.92498779296875, 369.59686279296875, 327.94097900390625, 380.5888671875)
  [6] Position: Rect(295.0369873046875, 411.59686279296875, 343.052978515625, 422.5888671875)
  [7] Position: Rect(391.28497314453125, 447.59686279296875, 439.30096435546875, 458.5888671875)
  [8] Position: Rect(302.3268127441406, 463.59686279296875, 350.2839660644531, 474.5888671875)


## 4. Anonymiser le PDF avec PyMuPDF

In [11]:
def anonymize_pdf(pdf_path: Path, noms_a_remplacer: list[str], nom_anonyme: str) -> bytes:
    """Remplace les noms dans le PDF et retourne les bytes du PDF anonymisé.

    Args:
        pdf_path: Chemin vers le PDF original
        noms_a_remplacer: Liste des variantes du nom à remplacer (triées par longueur desc)
        nom_anonyme: Nom de remplacement (ex: "ELEVE_001")

    Returns:
        Bytes du PDF anonymisé
    """
    # Optimisation pour éviter de supprimer du texte adjacent
    fitz.TOOLS.set_small_glyph_heights(True)

    doc = fitz.open(pdf_path)
    total_replacements = 0

    for page in doc:
        for nom in noms_a_remplacer:
            # Chercher toutes les occurrences de cette variante
            instances = page.search_for(nom)

            for rect in instances:
                # Ajouter une redaction avec le texte de remplacement
                page.add_redact_annot(
                    rect,
                    text=nom_anonyme,
                    fill=(1, 1, 1),  # Fond blanc
                    fontsize=10,    # Même taille que l'original
                )
                total_replacements += 1
                print(f"  Remplacé: '{nom}' à {rect}")

        # Appliquer les redactions
        page.apply_redactions()

    print(f"\nTotal remplacements: {total_replacements}")
    return doc.tobytes()


In [12]:
# Test avec les variantes NER
nom_anonyme = "ELEVE_ID_0001"

# Utiliser les variantes trouvées (priorité: variants > personnes_detectees > eleve_info)
if variants:
    noms_a_remplacer = variants
elif personnes_detectees:
    noms_a_remplacer = list(set(personnes_detectees))
elif eleve_info and eleve_info.get("nom_complet"):
    noms_a_remplacer = [eleve_info["nom_complet"]]
else:
    raise ValueError("Aucun nom trouvé à anonymiser. Vérifiez l'extraction PDF et le NER.")

print(f"Noms à remplacer ({len(noms_a_remplacer)}): {noms_a_remplacer}")
print(f"Remplacé par: {nom_anonyme}\n")

pdf_anonymise = anonymize_pdf(test_pdf, noms_a_remplacer, nom_anonyme)
print(f"\nTaille PDF anonymisé: {len(pdf_anonymise)} bytes")

Noms à remplacer (1): ['Marie Dupont']
Remplacé par: ELEVE_ID_0001

  Remplacé: 'Marie Dupont' à Rect(83.53968811035156, 123.1998062133789, 143.5596923828125, 133.19981384277344)
  Remplacé: 'Marie Dupont' à Rect(280.64276123046875, 215.93777465820312, 328.5696105957031, 223.93777465820312)
  Remplacé: 'Marie Dupont' à Rect(301.71697998046875, 257.9377746582031, 349.73297119140625, 265.9377746582031)
  Remplacé: 'Marie Dupont' à Rect(380.4129943847656, 345.9377746582031, 428.4289855957031, 353.9377746582031)
  Remplacé: 'Marie Dupont' à Rect(279.92498779296875, 371.9377746582031, 327.94097900390625, 379.9377746582031)
  Remplacé: 'Marie Dupont' à Rect(295.0369873046875, 413.9377746582031, 343.052978515625, 421.9377746582031)
  Remplacé: 'Marie Dupont' à Rect(391.28497314453125, 449.9377746582031, 439.30096435546875, 457.9377746582031)
  Remplacé: 'Marie Dupont' à Rect(302.3268127441406, 465.9377746582031, 350.2839660644531, 473.9377746582031)

Total remplacements: 8

Taille PDF anonymi

## 5. Vérifier le résultat

In [13]:
# Ouvrir le PDF anonymisé depuis les bytes
doc_anon = fitz.open(stream=pdf_anonymise, filetype="pdf")
page_anon = doc_anon[0]

# Vérifier que les noms originaux n'apparaissent plus
text_anon = page_anon.get_text()

print("=== Vérification de l'anonymisation ===\n")
for nom in noms_a_remplacer:
    present = nom in text_anon
    status = "❌ ENCORE PRÉSENT" if present else "✅ Supprimé"
    print(f"'{nom}': {status}")

print(f"\n'{nom_anonyme}': {'✅ Présent' if nom_anonyme in text_anon else '❌ Absent'}")

print(f"\n=== Texte anonymisé (500 premiers chars) ===\n{text_anon[:500]}")

=== Vérification de l'anonymisation ===

'Marie Dupont': ✅ Supprimé

'ELEVE_ID_0001': ✅ Présent

=== Texte anonymisé (500 premiers chars) ===
College Test
BULLETIN SCOLAIRE
Élève : 
Genre : Fille
Absences : 4 demi-journées (justifiées)
Engagements : Déléguée titulaire
Matière
Élève / Classe
Appréciation
Anglais LV1
15.21 / 10.83
Bons résultats. 
 fournit un travail régulier et sérieux à la maison tout comme
en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi!
Arts Plastiques
15.00 / 14.92
Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentrée.
EPS
8.00 / 12.79
Bilan très insuffisant, 
 n'a pa


In [14]:
# Sauvegarder le PDF anonymisé pour inspection visuelle
output_path = DATA_DIR / "processed" / "test_anonymise.pdf"
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "wb") as f:
    f.write(pdf_anonymise)

print(f"PDF anonymisé sauvegardé: {output_path}")
print("Ouvrez ce fichier pour vérifier visuellement le résultat.")

PDF anonymisé sauvegardé: c:\Users\Florent\Documents\data_science\chiron\data\processed\test_anonymise.pdf
Ouvrez ce fichier pour vérifier visuellement le résultat.


## 6. Test avec Mistral OCR (optionnel)

In [15]:
# Test avec Mistral OCR sur le PDF anonymisé
import base64
import json
import os

from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv(project_root / ".env")

# Vérifier que la clé API est configurée
api_key = os.getenv("MISTRAL_API_KEY") or os.getenv("MISTRAL_OCR_API_KEY")
if not api_key:
    print("⚠️ MISTRAL_API_KEY ou MISTRAL_OCR_API_KEY non configurée dans .env")
else:
    client = Mistral(api_key=api_key)

    # Encoder le PDF anonymisé en base64
    base64_pdf = base64.b64encode(pdf_anonymise).decode("utf-8")

    print("Envoi du PDF anonymisé à Mistral OCR...")
    response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": f"data:application/pdf;base64,{base64_pdf}"
        },
    )

    result = json.loads(response.model_dump_json())
    markdown = result["pages"][0]["markdown"]

    print("=" * 60)
    print("RÉSULTAT MISTRAL OCR SUR PDF ANONYMISÉ")
    print("=" * 60)
    print(markdown)

    # Vérifier que le nom original n'apparaît plus
    print("\n" + "=" * 60)
    print("VÉRIFICATION ANONYMISATION")
    print("=" * 60)
    for nom in noms_a_remplacer:
        if nom.lower() in markdown.lower():
            print(f"❌ '{nom}' ENCORE PRÉSENT dans le résultat OCR!")
        else:
            print(f"✅ '{nom}' absent du résultat OCR")

    if nom_anonyme in markdown:
        print(f"✅ '{nom_anonyme}' présent dans le résultat OCR")

Envoi du PDF anonymisé à Mistral OCR...
RÉSULTAT MISTRAL OCR SUR PDF ANONYMISÉ
# College Test

# BULLETIN SCOLAIRE

Élève : ELEVE_ID_0001

Genre : Fille

Absences : 4 demi-journées (justifiées)

Engagements : Déléguée titulaire

|  Matière | Élève / Classe | Appréciation  |
| --- | --- | --- |
|  Anglais LV1 | 15.21 / 10.83 | Bons résultats. ELEVE_ID_0001 fournit un travail régulier et sérieux à la maison tout comme en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi!  |
|  Arts Plastiques | 15.00 / 14.92 | Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentrée.  |
|  EPS | 8.00 / 12.79 | Bilan très insuffisant, ELEVE_ID_0001 n'a pas réussi à s'orienter avec efficacité. Elle a beaucoup marché et s'est dispersée avec son groupe. Son travail a manqué de sérieux et d'implication dans les défis à relever en course d'orientation. De sincères efforts sont attendus au prochain trimestre.  |
|  Éducation Musicale | 13.00 / 9.53 | C'est très bien quand

## 7. Conclusion

Si le test fonctionne :
- [ ] Intégrer `anonymize_pdf()` dans `src/document/`
- [ ] Modifier `MistralOCRParser` pour anonymiser avant envoi
- [ ] Stocker le mapping nom ↔ eleve_id